In [307]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from seaborn import pairplot
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.ticker as plticker

In [308]:
df = pd.read_csv('ML_epa_qbs_data.csv')
df = df[df['total_line'].notna()]
df

,Unnamed: 0,game_id,total_line,spread_line,home_score,away_score,result,total,season,week,...,posteam,defteam,home_qb_name,away_qb_name,off_epa,off_pass_epa,off_rush_epa,def_epa,def_pass_epa,def_rush_epa
0,1,2011_01_NO_GB,47.5,-4.5,42.0,34.0,-8.0,76.0,2011.0,1.0,...,GB,NO,Aaron Rodgers,Drew Brees,0.355356,0.554186,0.072405,0.160695,0.275887,-0.119056
1,2,2011_01_NO_GB,47.5,-4.5,42.0,34.0,-8.0,76.0,2011.0,1.0,...,NO,GB,Aaron Rodgers,Drew Brees,0.160695,0.275887,-0.119056,0.355356,0.554186,0.072405
2,3,2011_01_PIT_BAL,37.0,-1.0,35.0,7.0,-28.0,42.0,2011.0,1.0,...,BAL,PIT,Joe Flacco,Ben Roethlisberger,0.184658,0.345858,0.033533,-0.341011,-0.378989,-0.234197
3,4,2011_01_PIT_BAL,37.0,-1.0,35.0,7.0,-28.0,42.0,2011.0,1.0,...,PIT,BAL,Joe Flacco,Ben Roethlisberger,-0.341011,-0.378989,-0.234197,0.184658,0.345858,0.033533
4,5,2011_01_ATL_CHI,40.5,1.0,30.0,12.0,-18.0,42.0,2011.0,1.0,...,ATL,CHI,Jay Cutler,Matt Ryan,-0.262554,-0.290874,-0.166269,-0.055269,0.014545,-0.150940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5672,5673,2021_12_LAC_DEN,49.0,1.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5673,5674,2021_12_LA_GB,50.0,-1.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5674,5675,2021_12_MIN_SF,48.0,-2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5675,5676,2021_12_CLE_BAL,48.0,-4.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# going to group offense epa/play not by team but by starting qb. 
qb is the most important pos. in terms of wins/losses so by doing this we will avoid instances where a team is without their usual (or longtime) starting qb

In [309]:
home_qb_epa = df.groupby(['home_qb_name','posteam','home_team','season', 'week'], as_index=False)['off_epa'].mean()
away_qb_epa = df.groupby(['away_qb_name','posteam','away_team','season', 'week'], as_index=False)['off_epa'].mean()
defense_epa = df.groupby(['posteam', 'season', 'week'], as_index=False)['def_epa'].mean()

In [310]:
home_qb_epa = home_qb_epa[home_qb_epa['posteam'] == home_qb_epa['home_team']]

In [311]:
away_qb_epa = away_qb_epa[away_qb_epa['posteam'] == away_qb_epa['away_team']]

In [312]:
home_qb_epa.head(10)

,home_qb_name,posteam,home_team,season,week,off_epa
0,A.J. Feeley,LA,LA,2011.0,8.0,-0.008973
3,A.J. McCarron,CIN,CIN,2015.0,17.0,0.077666
4,A.J. McCarron,CIN,CIN,2015.0,18.0,-0.257034
5,A.J. McCarron,HOU,HOU,2019.0,17.0,-0.034610
42,Aaron Rodgers,GB,GB,2011.0,1.0,0.355356
43,Aaron Rodgers,GB,GB,2011.0,4.0,0.325846
44,Aaron Rodgers,GB,GB,2011.0,6.0,0.173265
45,Aaron Rodgers,GB,GB,2011.0,10.0,0.305516
46,Aaron Rodgers,GB,GB,2011.0,11.0,0.228829
47,Aaron Rodgers,GB,GB,2011.0,14.0,0.134226


In [313]:
away_qb_epa.head(10)

,away_qb_name,posteam,away_team,season,week,off_epa
1,A.J. Feeley,LA,LA,2011.0,7.0,-0.317436
2,A.J. Feeley,LA,LA,2011.0,13.0,-0.397723
4,A.J. McCarron,CIN,CIN,2015.0,15.0,-0.173835
5,A.J. McCarron,CIN,CIN,2015.0,16.0,0.085233
44,Aaron Rodgers,GB,GB,2011.0,2.0,0.339806
45,Aaron Rodgers,GB,GB,2011.0,3.0,0.032929
46,Aaron Rodgers,GB,GB,2011.0,5.0,0.194113
47,Aaron Rodgers,GB,GB,2011.0,7.0,0.236436
48,Aaron Rodgers,GB,GB,2011.0,9.0,0.269031
49,Aaron Rodgers,GB,GB,2011.0,12.0,0.230433


In [314]:
home_qb_epa['epa_shifted'] = home_qb_epa.groupby('home_qb_name')['off_epa'].shift()
away_qb_epa['epa_shifted'] = away_qb_epa.groupby('away_qb_name')['off_epa'].shift()
defense_epa['epa_shifted'] = defense_epa.groupby('posteam')['def_epa'].shift()

# now I have epa for each teams defense, and each qb when they play on the home and on the road... maybe should merge the home and road games for qbs? 

so.. whats happening here is that the qb epa is being shifted down from their latest start.  
ie.  since colt mccoy started a game in wk 10 of the 2021 season, he has a shifted epa present in the column for wk 1 of the 2011 season 

In [315]:
def dynamic_window_ewma(x):
    
    values = np.zeros(len(x))
    for i, (_, row) in enumerate(x.iterrows()):
        epa = x.epa_shifted[:i+1]
        if row.week > 4:
            values[i] = epa.ewm(min_periods=1, span=row.week).mean().values[-1]
        else:
            values[i] = epa.ewm(min_periods=1, span=4).mean().values[-1]
            
    return pd.Series(values, index=x.index)


home_qb_epa['ewma'] = home_qb_epa.groupby('home_qb_name')['epa_shifted'].transform(lambda x: x.ewm(min_periods=1, span=4).mean())
home_qb_epa['ewma_dynamic_window'] = home_qb_epa.groupby('home_qb_name').apply(dynamic_window_ewma).values

away_qb_epa['ewma'] = away_qb_epa.groupby('away_qb_name')['epa_shifted'].transform(lambda x: x.ewm(min_periods=1, span=4).mean())
away_qb_epa['ewma_dynamic_window'] = away_qb_epa.groupby('away_qb_name').apply(dynamic_window_ewma).values


defense_epa['ewma'] = defense_epa.groupby('posteam')['epa_shifted'].transform(lambda x: x.ewm(min_periods=1, span=4).mean())
defense_epa['ewma_dynamic_window'] = defense_epa.groupby('posteam').apply(dynamic_window_ewma).values

In [316]:
home_qb_epa = home_qb_epa.rename(columns={'home_qb_name':'qb'})
away_qb_epa = away_qb_epa.rename(columns={'away_qb_name':'qb'})

In [317]:
home_qb_epa.head()

,qb,posteam,home_team,season,week,off_epa,epa_shifted,ewma,ewma_dynamic_window
0,A.J. Feeley,LA,LA,2011.0,8.0,-0.008973,NaN,NaN,NaN
3,A.J. McCarron,CIN,CIN,2015.0,17.0,0.077666,NaN,NaN,NaN
4,A.J. McCarron,CIN,CIN,2015.0,18.0,-0.257034,0.077666,0.077666,0.077666
5,A.J. McCarron,HOU,HOU,2019.0,17.0,-0.034610,-0.257034,-0.131522,-0.099528
42,Aaron Rodgers,GB,GB,2011.0,1.0,0.355356,NaN,NaN,NaN


In [318]:
away_qb_epa.head()

,qb,posteam,away_team,season,week,off_epa,epa_shifted,ewma,ewma_dynamic_window
1,A.J. Feeley,LA,LA,2011.0,7.0,-0.317436,NaN,NaN,NaN
2,A.J. Feeley,LA,LA,2011.0,13.0,-0.397723,-0.317436,-0.317436,-0.317436
4,A.J. McCarron,CIN,CIN,2015.0,15.0,-0.173835,NaN,NaN,NaN
5,A.J. McCarron,CIN,CIN,2015.0,16.0,0.085233,-0.173835,-0.173835,-0.173835
44,Aaron Rodgers,GB,GB,2011.0,2.0,0.339806,NaN,NaN,NaN


# struggling to merge.. might have to backtrack at some point and calc qb epa without a home and road distinction-- tho this might still be useful as qbs likely have different splits on the home vs on the road

In [319]:
home_offense_defense = home_qb_epa.merge(defense_epa, on=['posteam', 'season', 'week'], suffixes=('_offense', '_defense')).rename(columns={"posteam": "team"})
away_offense_defense = away_qb_epa.merge(defense_epa, on=['posteam', 'season', 'week'], suffixes=('_offense', '_defense')).rename(columns={"posteam": "team"})

these two dataframes are the home team offense epa (by qb), and the defenses epa/play

In [320]:
home_offense_defense

,qb,team,home_team,season,week,off_epa,epa_shifted_offense,ewma_offense,ewma_dynamic_window_offense,def_epa,epa_shifted_defense,ewma_defense,ewma_dynamic_window_defense
0,A.J. Feeley,LA,LA,2011.0,8.0,-0.008973,NaN,NaN,NaN,-0.106512,0.239878,0.155591,0.135053
1,A.J. McCarron,CIN,CIN,2015.0,17.0,0.077666,NaN,NaN,NaN,-0.166730,0.074335,-0.049577,-0.054837
2,A.J. McCarron,CIN,CIN,2015.0,18.0,-0.257034,0.077666,0.077666,0.077666,-0.151540,-0.166730,-0.096438,-0.066238
3,A.J. McCarron,HOU,HOU,2019.0,17.0,-0.034610,-0.257034,-0.131522,-0.099528,0.501490,-0.204669,-0.005466,0.031927
4,Aaron Rodgers,GB,GB,2011.0,1.0,0.355356,NaN,NaN,NaN,0.160695,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,Zach Mettenberger,TEN,TEN,2014.0,14.0,-0.481742,0.092026,0.007364,-0.012793,0.019966,0.200760,0.131624,0.076940
2819,Zach Mettenberger,TEN,TEN,2015.0,7.0,-0.290406,-0.481742,-0.242180,-0.211006,-0.114029,0.236835,0.105122,0.071452
2820,Zach Mettenberger,TEN,TEN,2015.0,16.0,-0.362703,-0.290406,-0.264343,-0.219752,0.082666,0.132406,0.127712,0.069888
2821,Zach Wilson,NYJ,NYJ,2021.0,2.0,-0.320493,NaN,NaN,NaN,-0.054156,-0.015247,0.026008,0.026008


In [321]:
schedule = df[['season', 'week', 'home_team', 'away_team', 'home_score', 'away_score','spread_line','total_line', 'total']]\
    .drop_duplicates().reset_index(drop=True)\
    .assign(home_team_win = lambda x: (x.home_score > x.away_score).astype(int))\
    .assign(away_team_win = lambda x: (x.away_score > x.home_score).astype(int))

schedule.head()


,season,week,home_team,away_team,home_score,away_score,spread_line,total_line,total,home_team_win,away_team_win
0,2011.0,1.0,GB,NO,42.0,34.0,-4.5,47.5,76.0,1,0
1,2011.0,1.0,BAL,PIT,35.0,7.0,-1.0,37.0,42.0,1,0
2,2011.0,1.0,CHI,ATL,30.0,12.0,1.0,40.5,42.0,1,0
3,2011.0,1.0,CLE,CIN,17.0,27.0,-6.5,36.5,44.0,0,1
4,2011.0,1.0,HOU,IND,34.0,7.0,-9.0,44.0,41.0,1,0


In [322]:
home = schedule.merge(home_offense_defense).rename(columns={'qb':'home_qb'})
away = schedule.merge(away_offense_defense).rename(columns={'qb':'away_qb'})


In [323]:
games = home.merge(away, on=['season','week','home_team','away_team','home_score','away_score',
        'spread_line', 'total_line','total','home_team_win','away_team_win'],  suffixes=('_home', '_away')).drop_duplicates()
games


,season,week,home_team,away_team,home_score,away_score,spread_line,total_line,total,home_team_win,...,away_qb,team_away,off_epa_away,epa_shifted_offense_away,ewma_offense_away,ewma_dynamic_window_offense_away,def_epa_away,epa_shifted_defense_away,ewma_defense_away,ewma_dynamic_window_defense_away
0,2011.0,1.0,GB,NO,42.0,34.0,-4.5,47.5,76.0,1,...,Drew Brees,NO,0.160695,NaN,NaN,NaN,0.355356,NaN,NaN,NaN
1,2011.0,1.0,BAL,PIT,35.0,7.0,-1.0,37.0,42.0,1,...,Ben Roethlisberger,PIT,-0.341011,NaN,NaN,NaN,0.184658,NaN,NaN,NaN
2,2011.0,1.0,CHI,ATL,30.0,12.0,1.0,40.5,42.0,1,...,Matt Ryan,ATL,-0.262554,NaN,NaN,NaN,-0.055269,NaN,NaN,NaN
3,2011.0,1.0,CLE,CIN,17.0,27.0,-6.5,36.5,44.0,0,...,Andy Dalton,CIN,0.054779,-0.095739,-0.095739,-0.095739,-0.150630,NaN,NaN,NaN
4,2011.0,1.0,HOU,IND,34.0,7.0,-9.0,44.0,41.0,1,...,Kerry Collins,IND,-0.364075,NaN,NaN,NaN,-0.031704,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,2021.0,10.0,DEN,PHI,13.0,30.0,-1.0,44.5,43.0,0,...,Jalen Hurts,PHI,0.134323,0.233459,0.057044,0.012906,-0.260955,0.292557,0.081230,0.061985
2819,2021.0,10.0,GB,SEA,17.0,0.0,-3.0,49.5,17.0,1,...,Russell Wilson,SEA,-0.303277,-0.069162,0.033240,0.058154,0.064245,-0.149683,-0.064621,-0.008813
2820,2021.0,10.0,LV,KC,14.0,41.0,2.5,53.5,55.0,0,...,Patrick Mahomes,KC,0.390035,-0.100877,0.093085,0.140463,-0.163847,-0.005596,0.032090,0.085288
2821,2021.0,10.0,SF,LA,31.0,10.0,3.5,50.0,41.0,1,...,Matthew Stafford,LA,-0.215807,0.217188,0.171611,0.113279,0.157432,-0.077414,-0.064170,-0.050694


# might've got the data structured correctly, going to export the csv and inspect to check it out

In [324]:
games.to_csv('games.csv')

# it appears that for some reason the offensive epa isnt shifting over correctly whereas the defensive is? have to correct this before I run any ml models